# Pick-And-Place Dataset Generation (Python2)

## <div id="content">Contents</div>
1. <a href="#env" style="color:green">Setup Environment</a> 
2. <a href="#data" style="color:green">Dataset Generation</a>
3. <a href="#process" style="color:green">Data Filtering and Processing</a>

## <span id="env">Setup Environment</span>
<a href="#content">Return to Contents</a>

### <span id="setup-content">Section Contents</span>

1. <a href="#diagnostic">Diagnostics</a>
2. <a href="#packages">Import Packages & Utilities</a>
3. <a href="#launch">Launch Simulation Environment</a>

### <span id="diagnostic"> Diagnostic Information</span>
<span> <a href="#setup-content">Return to Section Contents</a> </span>

Notebook tested inside baxter.sh sudo environment. To access this use:
```
sudo ./baxter.sh sim
cd ..
jupyter notebook --allow-root
```
Otherwise Python path may not register properly.

In [ ]:
%%bash
if [ $(id -u) = 0 ];
then
    echo "You are root" # Should be root!
else
    echo "You do not have the right priviliges, you must be root"  
fi
python -V
source /opt/ros/kinetic/setup.bash
source ./workspace/devel/setup.bash

In [ ]:
# Check that engines are running
import ipyparallel as ipp # Run processes in parallel
c = ipp.Client()
print("Running Cluster with Engines: {}".format(c.ids))

### <span id="packages"> Python Packages </span>
<span> <a href="#setup-content">Return to Section Contents</a> </span>

In [ ]:
import sys
if sys.version_info[0] > 2:
    raise Exception("Must be using Python 2")

from IPython.display import clear_output # Clear Output in cell programmatically
from ipywidgets import IntProgress # Progress Bar
import os, sys, shutil #Advanced File Manipulation
from os import path
import glob
import matplotlib as mlt # Data Visualisation
import matplotlib.pyplot as plt
import cv2 # Interpret camera images
import pandas, numpy # Data Manipulation
from termcolor import colored # Colored Text

import subprocess # Used to run scripts in the background
import signal # Used to signal OS (e.g. to kill a process)
import csv # easily read and write to CSV files (used for ground truths)
import math # For trigonometry
import json # Effeciently store data structures

# Hyper-parameter Optimization Modules
import types
from types import * # Special Types (e.g. classes)
import inspect # Inspect method/class signatures
import pickle # Effeciently store classes to file


import getpass # For sudo-based tasks (hides input)
import re # Regular expressions
import random # For variation
import copy
import time # Allow python to wait
from enum import Enum

# ROS Packages
import rospy
import rospkg
import roslib
import roslaunch
import tf # Transformations
from tf.transformations import *
import std_msgs,gazebo_msgs,geometry_msgs,moveit_msgs
from geometry_msgs.msg import Pose, Point, Quaternion
from gazebo_msgs.msg import LinkStates
from gazebo_msgs.srv import DeleteModel, SpawnModel, GetModelState, GetWorldProperties
from std_msgs.msg import String
from sensor_msgs.msg import Image # Listen to baxter Cameras
import baxter_interface
import moveit_commander
import visual_interpreter
import retina

# >>> Utilities <<<
def define_pose(x,y,z,**kwargs):
    """Returns a Pose based on x,y,z and optionally o_x,o_y,o_z and o_w"""
    pose_target = geometry_msgs.msg.Pose()
    pose_target.position.x = x
    pose_target.position.y = y
    pose_target.position.z = z

    pose_target.orientation.x = kwargs.get("o_x",0.0)
    pose_target.orientation.y = kwargs.get("o_y",1.0)
    pose_target.orientation.z = kwargs.get("o_z",0.0)
    pose_target.orientation.w = kwargs.get("o_w",0.0)
    return pose_target

def get_rpy_quaternion(roll,pitch,yaw):
    """Converts human readable roll (about x-axis), pitch (about y-axis)
    and yaw (about z-axis) format to a 4D quaternion"""
    origin, xaxis, yaxis, zaxis = (0, 0, 0), (1, 0, 0), (0, 1, 0), (0, 0, 1)

    qz = quaternion_about_axis(yaw, zaxis)
    qy = quaternion_about_axis(pitch, yaxis)
    qx = quaternion_about_axis(roll, xaxis)

    q = quaternion_multiply(qz, qy)
    q = quaternion_multiply(q, qx)
    
    return q

def isclose(a,b,rel_tol=1e-9,abs_tol=0.0):
    """Returns whether 2 floats are equal with a relative and absolute 
    tolerance to prevent problems from floating point precision"""
    return abs(a-b) <= max( rel_tol * max(abs(a), abs(b)), abs_tol )

class color:
    """Defines special characters for decorating print statements"""
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
    
class label(Enum):
    """Defines labels for classifying pick-and-place images"""
    GRASP = 0
    BAD = 1
    NOT_READY = 2
    EMPTY = 3

### <span id="launch">Launch</span>
<span> <a href="#setup-content">Return to Section Contents</a> </span>

Setup the appropriate tools for simulating Baxter

**Terminal 1** (Gazebo)
> roslaunch baxter_gazebo baxter_world.launch

Wait for message "... Gravity Compensation was turned off"

**Terminal 2** (Joint Publishing)
> rosrun baxter_tools enable_robot.py -e

> rosrun baxter_interface joint_trajectory_action_server.py

Note the joint trajectory action server may require sudo priviliges

**Terminal 3** (Rviz)
> roslaunch baxter_moveit_config demo_baxter.launch left_electric_gripper:=true right_electric_gripper:=true

Motion planning will ALWAYS fail if electric grippers are not enabled

**Terminal 4** (Gazebo2Rviz)
> roslaunch gazebo2rviz gazebo2rviz.launch

Continuosly transfers objects in Gazebo Simulation to Rviz

In [ ]:
%%px --targets 0
import rospkg, rospy, os, roslaunch 

gazebo_path = rospkg.RosPack().get_path('baxter_gazebo')
launch_file_path = os.path.join(gazebo_path, "launch", "baxter_world.launch")

uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
roslaunch.configure_logging(uuid)
launch = roslaunch.parent.ROSLaunchParent(uuid, [launch_file_path])
launch.start()
rospy.loginfo("Started Gazebo")
print("{}Gazebo{} now running on engine 0".format(color.PURPLE,color.END))

In [ ]:
%%px --targets 1
import rospkg, rospy, os, roslaunch

package = 'baxter_tools'
executable = 'enable_robot.py'
node = roslaunch.core.Node(package, executable, args='-e')

launch = roslaunch.scriptapi.ROSLaunch()
launch.start()

process = launch.launch(node)

package = 'baxter_interface'
executable = 'joint_trajectory_action_server.py'
node = roslaunch.core.Node(package, executable)

launch = roslaunch.scriptapi.ROSLaunch()
launch.start()
process = launch.launch(node)
print("{}Joint Trajectory Action Server{} now running on engine 1".format(color.PURPLE,color.END))

In [ ]:
%%px --targets 2
import rospkg, rospy, os, roslaunch
rviz_path = rospkg.RosPack().get_path('baxter_moveit_config')
launch_file_path = os.path.join(rviz_path, "launch", "demo_baxter.launch")

#rospy.init_node('rviz', anonymous=True)
uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
roslaunch.configure_logging(uuid)
launch = roslaunch.parent.ROSLaunchParent(uuid, [launch_file_path])
launch.start()
rospy.loginfo("Started Rviz")
print("{}Rviz{} now running on engine 1".format(color.PURPLE,color.END))

In [ ]:
%%px --targets 3
# Note: Optional, transfers models from Gazebo to Rviz
import rospkg, rospy, os, roslaunch

gazebo2rviz_path = rospkg.RosPack().get_path('gazebo2rviz')
launch_file_path = os.path.join(gazebo2rviz_path,
                                "launch", "gazebo2rviz.launch")

#rospy.init_node('gazebo2rviz', anonymous=True)
uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
roslaunch.configure_logging(uuid)
launch = roslaunch.parent.ROSLaunchParent(uuid, [launch_file_path])

launch.start()
rospy.loginfo("Started Gazebo2Rviz")
print("{}Gazebo2Rviz{} now running on engine 1".format(color.PURPLE,color.END))

In [ ]:
%%px --targets 0
launch.shutdown()  # Shutdown Gazebo

In [ ]:
%%px --targets 1
process.stop()  # Stop Joint Trajectory Action Server

In [ ]:
%%px --targets 2
launch.shutdown()  # Shutdown Rviz

In [ ]:
%%px --targets 3 
launch.shutdown()  # Shutdown Gazebo2Rviz

## <span id="data">Dataset Generation</span>
<a href="#content">Return to Contents</a>

### <span id="dataset-content">Section Contents</span>

1. <a href="#spawn">Object Spawning</a>
    1. <a href="#spawn-table">Spawn Table</a>
    2. <a href="#spawn-block">Spawn Block</a>
    3. <a href="#spawn-custom">Spawn Custom Object</a>
2. <a href="#control">Dataset Generation Master</a>
    1. <a href="#control-manual">Manual</a>
    2. <a href="#control-automatic">Automatic</a>
3. <a href="#visual">Visualiation Tools</a>
    1. <a href="#visual-nodes">Display ROS nodes</a>
    2. <a href="#visual-camera">Get live camera feed</a>

In order to generate an appropriate dataset for Baxter to learn a generalized form of the pick-and-place task, a large set of examples is required. Since collecting data manually is resource-intensive, this has been automated as much as possible. Here hard-coded pick-and-place methods are used to move the robot to pick-up an object (of known location) in simulation. During this process, the video feed from Baxter's wrist ais recorded alongside useful metadata (pose, labels, etc.) under a clear naming scheme. 

The resulting dataset will contain sequences of images (ordered by their timestamp) where Baxter's wrist is seen to gradually approach the object until it is grasped. This data can be generated for a wide array of virtual objects to aid generalization, as the aim is to be able to pick-up unknown but gripper-compatible objects. 

The accompanying 'reward.csv' file contains all relevant metadata of frames from a specific iteration. Each iteration corresponds to 1 complete align-approach-grasp pipeline. The retract stage, which is used to determine whether the grasp was successful (i.e. does the object move along with the gripper), is not recorded as this information is redundant. The retino-cortical mapping will applied seperately in the next section to each of the filtered frames. Incorrect approaches, for instance where the motion planning fails, are also saved if these prove to make the final model more fault-tolerant.

Note that in the context of this project, only rigid objects are considered. However, there is room to expand the dataset to include deformable and articulated objects.

### <span id="spawn"> Object Spawning </span>
In order to switch out different objects and avoid overlap, the ObjectSpawner class provides the methods needed to spawn any object from the ~/.gazebo/models directory or the baxter_sim_examples ROS package.

It is possible to select any custom model via the private '_choose_model' interface, which is activated when no model_name is provided. This will recursively walk through the ~/.gazebo/models directory, allowing objects to be placed together in groups called 'categories'. Any selected models will be copied up to the root ~/.gazebo/models folder to allow Gazebo2Rviz to transfer the object from Gazebo to Rviz the object. An example model set can be found at [3DGems](http://data.nvision2.eecs.yorku.ca/3DGEMS/). 

BoundingBoxes are included to allow multiple objects to be spawned on the same table without overlap. However, since the dataset generator only spawns one at a time this can safely be ignored for most experiments.

It is possible to use additional model directories by specifying their full path (using os.path). However, these must be included in the Gazebo_Model_Path environmental variable using .bashrc:

```
export GAZEBO_MODEL_PATH=[YOUR_NEW_PATH]/models:$GAZEBO_MODEL_PATH
``` (See [Question](https://robotics.stackexchange.com/questions/1170/where-does-gazebo-set-the-gazebo-model-path-environment-variable))

In [ ]:
class BoundingBox(object):
    """Defines a cuboid volume to represent an object's bounding volume
    If only x is supplied, constructs a x*x*x cube
    If y is also supplied, constructs a x*y*x cuboid
    If y and z are supplied, constructs a x*y*z cuboid"""
    def __init__(self, x, y=None, z=None):  # x,y,z
        self.x = x
        self.y = x if y is None else y
        self.z = x if z is None and y is None else (y if z is None else z)


class ObjectSpawner(object):
    """Provides convenient API for spawning any objects in the Gazebo model directory
    
    Attributes:
        -- Provided at initalization --
        reference_frame: coordinates are used relative to ref frame [default: 'world']
        bounding_box: defines a bounding volume for the object [default: 0.04*0.04*0.04]
        model_name: if not provided, interactively choose model [default: None]
        model_directory: if not provided, use default directory [default: None]
        verbose: How much daignostic info to print [default: 1]
        -- Generated --
        nested_dir: Gazebo models can only be used if placed at root of Gazebo model directory"""
    instances = {}

    def __init__(self, reference_frame="world", bounding_box=BoundingBox(0.04),
                 model_name=None, model_directory=None,
                 verbose=1):
        self.reference_frame = reference_frame
        self.bounding_box = bounding_box

        self.default_dir = path.join(os.getcwd(), "workspace", "src",
                                     "baxter_simulator",
                                     "baxter_sim_examples", "models")
        if model_directory is None:
            self.model_dir = self.default_dir
        else:
            self.model_dir = model_directory

        if model_name is None:
            self.model_name = self._choose_model()
        else:
            self.model_name = model_name

        # True if model is found in root model_path
        if not hasattr(self, "nested_dir"):
            self.nested_dir = self.model_dir

        self.verbose = verbose

    def spawn_model(self, pose):
        """Spawns a model at the target pose (position and orientation).
        
        If Spawning service failed: Returns None
        Else: Returns name of generated object and its real pose"""
        
        model_path = path.join(self.nested_dir, self.model_name, "model.sdf")
        with open(model_path, "r") as f:
            xml = f.read()

        if self.verbose >= 3:
            print(">> Loaded Model {} <<<\n{}".format(self.model_name, xml))

        rospy.wait_for_service('/gazebo/spawn_sdf_model')
        try:
            spawn_sdf = rospy.ServiceProxy('/gazebo/spawn_sdf_model',
                                           SpawnModel)

            # Enforce unique names
            if len(self.instances) == 0:
                name = self.model_name
            else:
                "{}_{}".format(self.model_name, len(self.instances))

            spawn_sdf(name, xml.replace("\n", ""),
                      "/", pose, self.reference_frame)

            self.instances[name] = pose
            return name, pose
        except rospy.ServiceException, e:
            rospy.logerr("Spawn {} failed: {}".format(self.model_name, e))
            return None

    def _choose_model(self):
        """Interactively select model by traversing Gazebo model subdirectories
        Copies chosen model to top of Gazebo Model path so that it is accessible
        Returns the chosen model name"""
        # Only descend into N levels of file path tree
        def walklevel(some_dir, level=1):
            some_dir = some_dir.rstrip(os.path.sep)
            assert os.path.isdir(some_dir)
            num_sep = some_dir.count(os.path.sep)
            for root, dirs, files in os.walk(some_dir):
                yield root, dirs, files
                num_sep_this = root.count(os.path.sep)
                if num_sep + level <= num_sep_this:
                    del dirs[:]

        # Get all VALID folders with mesh options
        def get_listdir(path, verbose=True):
            valid_dirs = {}
            count = 0
            for dir_name, child_dirs, files in walklevel(path, level=1):
                if child_dirs in [[], ['meshes'], ['materials', 'meshes']]:
                    continue
                else:
                    if verbose: print("{}: {}".format(count, dir_name))
                    valid_dirs[count] = dir_name
                    count += 1
            return valid_dirs

        # Ask user to pick directory by number
        def select_by_number(options, option_desc):
            while True:
                resp = raw_input("Type no of desired {}".format(option_desc))
                if re.match("\d+", resp):
                    if int(resp) in options.keys():
                        return options[int(resp)]  # Selected option
                print("Please pick desired {} by number".format(option_desc))

        if self.model_dir != self.default_dir:
            valid_folders = get_listdir(self.model_dir)
            category = select_by_number(valid_folders, "Category")
            self.nested_dir = path.join(self.model_dir, category)
            valid_models = os.listdir(self.nested_dir)
        else:  # Default Directory does not have categories
            valid_models = os.listdir(self.model_dir)

        # Final Choice: One Specific Model
        valid_models = dict(enumerate(valid_models, start=0))
        for key, value in valid_models.items():
            print("{}: {}".format(key, value))
        model_name = select_by_number(valid_models, "Model")

        if self.verbose >= 1: print("You selected: {}".format(model_name))

        return model_name

    def spawn_on_table(self, spawn=True, random_face=True,
                       table_bbox = BoundingBox(0.32, 0.5, 0.7825),
                       table_offset = Point(0.38, -0.1, 0.0)):
        """Spawns an object on a table's surface with random location and yaw. 
        
        Attributes:
            spawn: Whether to actually spawn the object [default:True]
            random_face: Whether to vary which face is front-facing (24 possibilities) [default: True]
            table_bbox: Defines table's bounding box, ensure this is reachable [default: 0.32*0.5*0.7825]
            table_offset: Where table is placed relative to world origin (0,0,0)
        Returns:
            Unique name in simulation, overhead RPY and real pose
        """
        x = random.uniform(table_offset.x, table_offset.x + table_bbox.x)  # Default: 0.38->0.7
        y = random.uniform(table_offset.y, table_offset.y + table_bbox.y)  # Default: -0.1->0.4)
        z = table_offset.z + table_bbox.z  # Default: 0.7825
        fixed_rotations = numpy.arange(0, 2*numpy.pi, numpy.pi/2)
        if random_face:
            roll, pitch, yaw = random.choice(fixed_rotations), \
                               random.choice(fixed_rotations), \
                               random.uniform(0, 2*numpy.pi)
        else:
            roll, pitch, yaw = numpy.pi, 0, random.uniform(0, 2*numpy.pi)

        q = get_rpy_quaternion(roll, pitch, yaw)

        pose = define_pose(x=x, y=y, z=z, o_x=q[0],
                           o_y=q[1], o_z=q[2], o_w=q[3])

        # ToDo: Use bounding boxes to avoid overlap

        name = None
        if spawn:
            name, _ = self.spawn_model(pose)

            # Make model available at top level of Model directory
            root_model_dir = path.join(self.model_dir, self.model_name)
            if not path.exists(root_model_dir):
                nested_model_dir = path.join(self.nested_dir, self.model_name)
                shutil.copytree(nested_model_dir, root_model_dir)

            if self.verbose >= 2:
                print("Spawned {} at (x={},y={},z={})".format(self.model_name,
                                                              x, y, z))

        return name if name is not None else "EMPTY", (numpy.pi, 0, yaw), pose

    def delete_models(self, instance_name=None):
        """Use class-wide instances attribute to delete models. 
        instance_name: if specified, deletes specific model [default: None]"""
        try:
            delete_model = rospy.ServiceProxy("/gazebo/delete_model",
                                              DeleteModel)
            if instance_name is None:
                for instance_name in self.instances.keys():
                    delete_model(instance_name)
                    del self.instances[instance_name]
            elif instance_name in self.instances:
                delete_model(instance_name)
                del self.instances[instance_name]
        except rospy.ServiceException as e:
            rospy.loginfo("Delete Model failed: {}\n{}".format(e, instances))


Object Spawner signature:
* **reference_frame** (OPTIONAL): Reference Frame in which to spawn the object. Default: "world"
* **bounding_box** (OPTIONAL): Defines overall size of the model using BoundingBox class. Default: BoundingBox(0.04)
* **model_name** (OPTIONAL): General name of model, if not provided opens user selection wizard. default: none
* **model_directory** (OPTIONAL): Root directory in ROS path where models can be found, uses WORKSPACE_PATH/ baxter_sim_examples/models if not provided. Default: None
* **verbose** (OPTIONAL): Whether to print extra text. Default: False
The ObjectSpawner keeps track of ALL spawned_objects across every ObjectSpawner instance generated. This means if you use delete_models() it will remove ALL objects in Gazebo that have been spawned using the Object Spawner.

#### <span id="spawn-table">Spawn Table</span>
<span> <a href="#dataset-content"> Return to Section Contents </a> </span>
    
If the table spawner fails, you may need to generate an sdf file first using [pysdf](http://wiki.ros.org/pysdf).

The DataSet generator has a table_spawner built in, hence only use this cell for manual runs.

In [ ]:
table_spawner = ObjectSpawner(reference_frame="world",
                              bounding_box=BoundingBox(0.913, 0.913, 0.7825),
                              model_name="cafe_table")


TableTop is 0.914m by 0.914m placed at x=0.8, y=0.0,z=0.7747. 
Hence valid area for spawning objects:
* x = 0.343 m to x = 1.257 m
* y = -0.257 m to x = 0.257 m
* z 0.7825

In [ ]:
table_spawner.spawn_model(Pose(position=Point(x=0.0, y=1.0, z=0.0)))

In [ ]:
table_spawner.spawn_model(Pose(position=Point(x=0.8, y=0.0, z=0.0)))

In [ ]:
table_spawner.delete_models()

#### <span id="spawn-block">Spawn Block</span>
<span> <a href="#dataset-content"> Return to Section Contents </a> </span>

If the block spawner fails, you may need to generate an sdf file first using [pysdf](http://wiki.ros.org/pysdf).

The DataSet generator has a spawner (default=block) built in, hence only use this cell for manual runs.

In [ ]:
block_spawner = ObjectSpawner(reference_frame="world",
                              bounding_box=BoundingBox(0.04),
                              model_name="block")


In [ ]:
block_spawner.spawn_model(Pose(position=Point(x=0.8, y=0.0, z=0.7825)))

In [ ]:
block_spawner.delete_models()

#### <span id="spawn-custom"> Spawn Custom model Block </span>
<span> <a href="#dataset-content"> Return to Section Contents </a> </span>

The DataSet generator has a custom spawner (default=block) built in, hence only use this cell for manual runs.

Please follow instructions for [3DGEMS](http://data.nvision2.eecs.yorku.ca/3DGEMS/) under the Project_Setup manual to get the appropriate sdf file here. <br>
Not all models will have their physics properties specified.

In [ ]:
custom_model_path = path.join(os.path.expanduser('~'), ".gazebo", "models")
custom_spawner = ObjectSpawner(reference_frame="world",
                               model_directory=custom_model_path)
clear_output()
print("Created {} spawner".format(custom_spawner.model_name))

In [ ]:
custom_spawner.spawn_on_table()

In [ ]:
custom_spawner.delete_models()

### <span id="control">Dataset Generation Master </span>
<span> <a href="#dataset-content">Return to Section Contents</a> </span>

In [ ]:
class MoveBaxter(object):
    """
    MoveBaxter automates simple pick-and-place tasks using the Baxter Robot.

    Attributes:
        -- Provided at initialization --
        id (string): Used for identifying the MoveBaxter instance when generating datasets
        limb_name (string): Choice between 'left' and 'right' MoveGroup [default: 'left']
        model_pattern (string): Name of spawnable model (must be in .gazebo/models or baxter_sim_examples/models) [default:'block']
        max_attempts (int): Maximum number of attempts to try to find a motion plan [default: 100]
        verbose (int): Print extra diagnostic information (0-None,1-Minimal,2-Moderate, 3-All)
        planning_time (float): Time in seconds allowed per planning attempt [default: 1.0 seconds]
        tolerance (float): Variation allowed in planning target joint, position and orientation of end effector

        -- Generated --
        spawner (ObjectSpawner): Class used to spawn Object of model_pattern on Table
        gripper (baxter_interface): Used to open and close Baxter's gripper
        display_trajectory_publisher (rospy Publisher): Publishes motion plans to Rviz for visualisation
        get_model_state (rospy Service): Allows for pose retrieval of specific links
        tf_listener (TransformListener): Provides reliable way to get pose of MoveGroup
        robot (RobotComander): For diagnostics, allows for a complete recording of Robot Status
        group (MoveGroupCommander): Target arm to be used for performing pick_and_place task
        gt_path (string): Path to ground_truth csv file
    """

    def __init__(self, id, limb_name="left", model_pattern="block", max_attempts=100,
                 planning_time=1.0, tolerance=0.5, verbose=1):
        self.id = id
        self.limb_name = limb_name
        self.verbose = verbose
        self.max_attempts = max_attempts

        # Which Model to Spawn on Table for picking
        self.model_pattern = model_pattern
        self.spawner = ObjectSpawner(reference_frame="world",bounding_box=BoundingBox(0.04),
                              model_name=model_pattern)
        self.table_spawner = ObjectSpawner(reference_frame="world",bounding_box=BoundingBox(0.913,0.913,0.7825),
                              model_name="cafe_table")

        self.gripper = baxter_interface.Gripper(limb_name)

        baxter_interface.RobotEnable(baxter_interface.CHECK_VERSION).state().enabled

        if verbose >= 2: print("Enabled Robot")

        self.stage_pub = rospy.Publisher('pick_stage_publisher', String, queue_size=10)
        self.display_trajectory_publisher = rospy.Publisher("/move_group/display_planned_path",
                                                            moveit_msgs.msg.DisplayTrajectory,
                                                            queue_size=20)
        self.get_model_state = rospy.ServiceProxy('/gazebo/get_model_state', GetModelState)
        self.tf_listener = tf.TransformListener()

        self.robot = moveit_commander.RobotCommander()

        self.tolerance = tolerance
        self.group = moveit_commander.MoveGroupCommander("left_arm")
        self.group.allow_replanning(True)
        self.group.set_planning_time(planning_time) # Seconds
        self.group.set_goal_tolerance(tolerance) # Joint, Position, Orientation 
        self.group.set_goal_position_tolerance(tolerance/10)

        self.gt_path=os.path.join("workspace","data","{}_ground_truths.csv".format(self.id))

        if verbose >= 1: print("Successfully initialized MoveBaxter")

    def get_diagnostics(self):
        """
        Provides a End Effector, Tolerance and (Optionally) Robot Status update
        
        Parameters:
            verbose (bool): Override class verbosity to print complete Robot State [default: False]
        """
        print("[MoveBaxter] Planning Frame: {}".format(self.group.get_planning_frame()))
        print("[MoveBaxter] End Effector: {}\n{}".format(self.group.get_end_effector_link(),
                                                         self.get_current_pose()))
        print("[MoveBaxter] Goal Tolerance:{}".format(self.group.get_goal_tolerance()))
        if self.verbose >= 3: print(">>> Current Robot State <<<\n{}".format(self.robot.get_current_state()))

    def get_object_state(self, model_name=None):
        """
        Try to get status of a object in simulation.
        
        Parameters:
            model_name (String): if None search all spawned instances [default:None]
            
        Returns:
            GetModelState msg (if object state was found)
            None (if failed to get object state)"""
        rospy.wait_for_service("/gazebo/get_model_state")
        success = False
        if model_name is None:
            for i in range(len(self.spawner.instances)):
                # If multiple instances of a model are spawned, Gazebo appends _NUM 
                model_name = self.model_pattern + ("_{}".format(i) if i > 0 else "")
                result = self.get_model_state(model_name,"")
                success = result.success
                if success: break
        else:
            result = self.get_model_state(model_name, "")
            if result is not None: success = result.success
        if success is False:
            return None
        else:
            return result # GetModelState Msg

    def grasp_verification(self, original_pose, is_full):
        """
        Checks whether target_object has changed its z-position as a result of a pick-attempt
        
        Parameters:
            original_pose (Pose): Pose of object when it was originally spawned
            
        Returns:
            None (if target link could not be found)
            False (if object did not move vertically or fell)
            True (if object did move vertically)
            old_z, new_z (float, position along z-axis. Allows for cross-checking)
        """
        new_z, old_z = None,original_pose.position.z
        obj_z = self.spawner.bounding_box.z

        # Empty state
        if not is_full: return False, original_pose, original_pose

        res = self.get_object_state()
        if res is None: return None, None, None

        # Compare original with new
        new_pose = res.pose
        new_z = new_pose.position.z
        if isclose(old_z, new_z, rel_tol=1e-9, abs_tol=self.tolerance/10) or new_z < old_z + obj_z:
            return False, original_pose, new_pose
        else:
            return True, original_pose, new_pose

    def object_in_view(self, model_name = None, table_z = 0.7825):
        """
        Checks whether a specific object is in the camera's viewing frustum.
        Disclaimer: Not currently accurate for objects at distances <= 0.1
        
        Parameters:
            model_name (String): If not provided, find any spawned instance [default: None]
            table_z (float): Height of table [default: 0.7825]
            
        Returns:
            True (if object is inside viewing frustum)
            False (if object is outside of viewing frustum)
            None (if failed to get object state)
        """

        res = self.get_object_state(model_name=model_name)
        if res is None: return None  # Object DNE

        obj_pos = res.pose.position
        bbox = move_baxter.spawner.bounding_box

        # Get Camera Position
        pos, _ = self.get_current_pose()
        camera_offset = (0.03825, 0.012, 0.015355) # From urdf
        # Adjust for Camera's offset from Gripper Position
        pos[0] += camera_offset[0]; pos[1] += camera_offset[1]; pos[2] += camera_offset[2]

        lim = pos[2] - table_z

        if self.verbose >= 3:
            print(">>> Checking if {} Object in view <<<".format(model_name if model_name is not None else ""))
            print("Camera Position: {}".format(pos))
            print("Limit (+-): {}".format(lim))
            print("Object Position: {}+-{},{}+-{}".format(obj_pos.x, bbox.x, obj_pos.y, bbox.y))
            print("Table Height: {}".format(table_z))

        if (((pos[0] - lim < abs(obj_pos.x + bbox.x) < pos[0] + lim) or 
            (pos[0] - lim < abs(obj_pos.x - bbox.x) < pos[0] + lim)) and 
            ((pos[1] - lim < abs(obj_pos.y + bbox.y) < pos[1] + lim) or 
            (pos[1] - lim < abs(obj_pos.y - bbox.y) < pos[1] + lim))):
            return True # In View
        else:
            return False # Not In View

    def reset_models(self, spawn=True):
        """
        Remove all object instances, but NOT the table
        Then generate a new version of the object
        
        Parameters:
            spawn (bool): Whether to spawn a new object afterwards
        Returns:
            Pose (to actual item if spawn=True, to empty spot if spawn=False)
        """
        if self.verbose >= 1: print("Resetting all models with{} new spawn".format("" if spawn else "out"))
        self.spawner.delete_models()
        self.table_spawner.delete_models()
        time.sleep(0.1)
        self.table_spawner.spawn_model(Pose(position=Point(x=0.8, y=0.0, z=0.0)))
        if spawn:
            time.sleep(0.1)
            return self.spawner.spawn_on_table() # Returns: name, RPY, Pose
        else: 
            # Returns: 'EMPTY',RPY,Pose (no object at returned Pose!)
            return self.spawner.spawn_on_table(spawn=False)

    def loop_training(self,iterations=10, hover_distance=0.3, no_empty_state=False):
        """
        Generates a image-sequence dataset with labels for a randomized pick task.
        
        Parameters:
            iterations (int): How many times to pick up the object 
            hover_distance (float): What height to hover above the object (must clear object)
        """
        self.return_to_home_pose(open_gripper=True)

        for i in range(iterations):
            session_id = "{}_iteration_{}".format(self.id, i)

            # >>> Reset : New Iteration <<<
            spawn_or_not = True if no_empty_state else False#bool(random.getrandbits(1))
            object_name, RPY, spawned_pose = self.reset_models(spawn=spawn_or_not)
            self.open_gripper()

            hover_pose = define_pose(spawned_pose.position.x,
                                    spawned_pose.position.y,
                                    spawned_pose.position.z)
            # Note: Spawned Pose quaternion will NOT match overhead gripper quaternion
            q = get_rpy_quaternion(*RPY) # Hence we use the RPY of the upright-yaw component only
            target_pose = define_pose(spawned_pose.position.x,
                                      spawned_pose.position.y,
                                      spawned_pose.position.z,
                                      o_x=q[0], o_y=q[1], o_z=q[2], o_w=q[3])

            # >>> Start Reward Publisher <<<
            time_start = time.time()
            reward_pub = subprocess.Popen(['bash', 'scripts/run_metadata_publisher.sh',
                                           '-l', 'left',
                                           '-o',object_name,
                                           '-s', str(session_id),
                                           '-e', 'false' if spawn_or_not else 'true'])
            time.sleep(1) # Wait for Publisher to Initialize

            # >>> Start Recording <<<
            recorder = subprocess.Popen(['bash', 'scripts/run_recorder.sh',
                                         'left', str(session_id)])
            time.sleep(3) # Wait for Recorder to Initialize

            # >>> Hover over Target <<<
            self.stage_pub.publish("SEARCH")
            self.hover_over_target(hover_pose, hover_distance=hover_distance)
            time.sleep(1)

            # >>> Pick Target <<<
            self.stage_pub.publish("APPROACH")
            self.move_to_target(target_pose, direct=True)
            # Publish: In Graspable/Ungraspable Stage
            self.stage_pub.publish("GRASP")
            time.sleep(1) # Ensure graspable position is captured
            self.close_gripper()
            time.sleep(1) # Ensure grasped object is captured

            # >>> Stop Recording <<<
            kill = subprocess.Popen(['bash', 'scripts/kill_recorder.sh', 
                                     session_id])
            kill.wait(); recorder.wait() # Wait for node to shutdown cleanly
            self.stage_pub.publish("IDLE")

            # >>> Stop Reward Publisher <<<
            kill = subprocess.Popen(['bash', 'scripts/kill_metadata_publisher.sh', 
                                     session_id])
            kill.wait(); reward_pub.wait()

            duration = time.time() - time_start # in seconds

            # >>> Feedback <<<
            if self.verbose >= 3:
                print("---- Iteration: {}, Session: {} ----".format(self.id, i))
                print(">>> Spawned Pose <<<\n{}".format(spawned_pose))
                print(">>> Hover Pose <<<\n{}".format(hover_pose))

            # >>> Check Grasp Success <<<
            self.hover_over_target(hover_pose, 
                                   hover_distance=hover_distance)
            rospy.sleep(0.5) # Ensure end position is captured

            ground_truth_label, old_pose, new_pose = self.grasp_verification(spawned_pose, 
                                                                           spawn_or_not)
            if self.verbose >= 1:
                print("Grasp {}".format("Successful" if ground_truth_label else "Unsuccessful"))

            # >>> Reset: Return Home <<< 
            self.return_to_home_pose(open_gripper=False)

            # >>> Save Grasp Success and MetaData <<<
            with open(self.gt_path, 'a') as f:
                fieldnames = ["data_id", "iteration", "grasp_success", "duration",
                              "spawned_position", "end_position",
                              "spawned_orientation", "end_orientation"]
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                if f.tell() == 0: # Header
                    writer.writeheader()
                writer.writerow({'data_id': self.id, 'iteration': i,
                                 'grasp_success': ground_truth_label, 'duration': duration,
                                 'spawned_position': old_pose.position,
                                 'end_position': new_pose.position,
                                 'spawned_orientation':old_pose.orientation,
                                 'end_orientation':new_pose.orientation})

        # Done
        self.reset_models(spawn=False)
        self.return_to_home_pose(open_gripper=True)

    def display_trajectory(self, plan):
        """Publishes motion plan to Rviz for visualisation"""
        display_trajectory = moveit_msgs.msg.DisplayTrajectory()
        display_trajectory.trajectory_start = self.robot.get_current_state()
        display_trajectory.trajectory.append(plan)
        self.display_trajectory_publisher.publish(display_trajectory)

    def get_current_pose(self,verbose=False):
        """Returns current pose (position/orientation) of gripper"""
        current_pose = self.tf_listener.lookupTransform("base",
                                                        self.group.get_end_effector_link(),
                                                        rospy.Time(0))
        if verbose >= 2: print("Current End Effector Pose:\n{}".format(current_pose))
        return current_pose

    def move_to_target(self, target, direct=False):
        """
        Attempt up to self.max_attempts to move the end effector to the target pose.
        
        Paramaters:
            target (Pose): Desired position and orientation for end effector
        """
        if not isinstance(target, geometry_msgs.msg.Pose):
            raise TypeError("Target must be of type Pose")
        #self.group.clear_pose_targets()
        self.group.set_pose_target(target)
        result = False; fraction = 1.0
        for attempt in range(0, self.max_attempts):
            if direct:
                (plan, fraction) = self.group.compute_cartesian_path(
                                       [target],   # waypoints to follow
                                       0.01,       # eef_step
                                       0.0)        # jump_threshold
            else:
                plan = self.group.plan()

            if plan is not None and len(plan.joint_trajectory.points)>0 and isclose(fraction, 1.0):
                if self.verbose >= 2: print("Attempt {}, motion plan FOUND".format(attempt))
                self.display_trajectory(plan)
                result = self.group.execute(plan,wait=True)
                if result:
                    if self.verbose >= 1: print("Motion Plan Execution Succeeded")
                    break
            else:
                if self.verbose >= 2: print("Attempt {}, motion plan NOT FOUND".format(attempt))
        if result and plan is not None:
            rospy.sleep(1.0)
            self.group.stop()
            self.group.clear_pose_targets()
        else:
            if self.verbose >= 1: print("No Motion Plan Found in {} attempts".format(self.max_attempts))
                         
        return result

    def open_gripper(self):
        """Opens gripper and waits 1.0 seconds"""
        if self.verbose >= 2: print("Opening {} gripper...".format(self.limb_name))
        self.gripper.open()
        rospy.sleep(1.0)

    def close_gripper(self):
        """Closes gripper and waits 1.0 seconds"""
        if self.verbose >= 2: print("Closing {} gripper...".format(self.limb_name))
        self.gripper.close()
        rospy.sleep(1.0)

    def return_to_home_pose(self, open_gripper=True):
        """Returns arm to neutral home pose 
        
        Parameters:
            open_gripper (bool): Whether to open gripper
        """

        if open_gripper: self.open_gripper()
        pose_target = define_pose(x=0.46,
                                  y=0.8 if self.limb_name == "left" else -0.8,
                                  z=1.08)  # 0.69
        success = self.move_to_target(pose_target)
        if success and self.verbose >= 1:
            print("Moved the {} arm to start pose...".format(self.limb_name))
        rospy.sleep(1.0)

    def hover_over_target(self, supplied_pose, hover_distance=0.3):
        """Hovers gripper over object WITHOUT rotational alignment"""

        if not isinstance(supplied_pose, geometry_msgs.msg.Pose):  # Pose
            raise TypeError("Target was not of type Pose: {}".format(type(target)))
        target_pose = copy.deepcopy(supplied_pose)
        target_pose.position.z += hover_distance
        
        success = self.move_to_target(target_pose)
        if success and self.verbose >= 1: 
            print("Moving the {} arm over target".format(self.limb_name))

#### Automatic Dataset Generation

Automatically generate a custom dataset with 1000 iterations (approx 2GB). Will repeatedly spawn, align, approach, grasp and then retract. MoveBaxter will spawn appropriate publishers and subscribers to save information to the **PROJECT_DIR/workspace/data** directory.

Parameters that may be customized:
* **model_name**: Which object to spawn from ~/.gazebo/models or baxter_sim_examples directories
* **tolerance**: How accurate motion planning should be (smaller is more accurate, but fails more often)
* **limb_name**: Whether to use right or left planning group
* **verbose**: How much diagnostic information to print during operation

In [ ]:
print(MoveBaxter.__doc__)

**Note**: Gazebo, Rviz and JointTrajectoryActionServer must be running

In [ ]:
run_name = raw_input("Enter a name for the current Run >>>")

moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node("MoveBaxter_Interface", anonymous=True)
move_baxter = MoveBaxter(run_name, limb_name="left", tolerance=0.01, verbose=1)  
move_baxter.open_gripper()

In [ ]:
move_baxter.loop_training(iterations=1000, no_empty_state=False)

#### Manual Data Generator
MoveBaxter also allows manual control of the robot, this is primarily useful for testing new features or evaluating generated data.

In [ ]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node("MoveBaxter_Interface", anonymous=True)
move_baxter = MoveBaxter("Experimental", limb_name="left", tolerance=0.01, verbose=1)  
move_baxter.open_gripper()

**Reset**: Spawn 1 object (default=block) in a fresh environment. Then retrieve its pose for the alignment stage.

In [ ]:
name,RPY,spawned_pose = move_baxter.reset_models(spawn=True)
hover_pose = define_pose(spawned_pose.position.x,
                        spawned_pose.position.y,
                        spawned_pose.position.z)

**Target**: Set orientation (adjusted based on the yaw of the object) for the final pick action.

In [ ]:
q = get_rpy_quaternion(*RPY) # Hence we use the RPY of the upright-yaw component only
target_pose = define_pose(spawned_pose.position.x,
                          spawned_pose.position.y,
                          spawned_pose.position.z,
                          o_x=q[0],o_y=q[1],o_z=q[2],o_w=q[3])

**Home**: Return to Home Pose, verify that object is not in view.

In [ ]:
move_baxter.return_to_home_pose()
move_baxter.object_in_view()

**Hover**: Move to align above the object (overhead orientation)

In [ ]:
move_baxter.hover_over_target(hover_pose)
move_baxter.object_in_view()

**Diagnostic**: Get gripper and object poses to deduce the distance between them (used to calculate reward)

In [ ]:
position,orientation=move_baxter.get_current_pose()
print(type(Point(*position)),type(Quaternion(*orientation)))
gripper_pose = Pose(Point(*position),Quaternion(*orientation))
object_pose = move_baxter.get_model_state("block","").pose

x2,y2,z2 = object_pose.position.x, object_pose.position.y, object_pose.position.z
x1,y1,z1 = gripper_pose.position.x, gripper_pose.position.y, gripper_pose.position.z

dx,dy,dz = x2-x1,y2-y1,z2-z1
print("dx:{},dy:{},dz:{}".format(dx,dy,dz))
distance = math.sqrt(math.pow(dz,2)+math.pow(dy,2)+math.pow(dx,2))
print("Distance: {}".format(distance))
print(type(object_pose),type(gripper_pose))
#object_pose.inve

**Pick**: Move from hover position to move gripper around graspable points on spawned object. Use 'compute_cartesian_path' to plot a direct path.

In [ ]:
move_baxter.move_to_target(target_pose,direct=True)

**Grasp**: Close grippers around object

In [ ]:
move_baxter.close_gripper()

**Retract & Verify**: Return to home pose (with object in grasp) and check whether object was grasped successfully

In [ ]:
move_baxter.hover_over_target(hover_pose)
grasp_successful = move_baxter.grasp_verification(original_pose=spawned_pose)
print("Grasp {}".format("successful" if grasp_successful else "unsuccessful")

### <span id="visual">Visualisation Tools</span>

#### <span id="visual-nodes">Display ROS Nodes<span>
<span> <a href="#dataset-content"> Return to Section Contents </a> </span>
     
ROS employs a Publisher-Subscriber pattern for asynchronous communication between nodes. The "rqt_graph" node examines all published topics and their subscribers in order to illustrate the network of connections between different nodes. This can help diagnose what information a particular node can act on.

In [ ]:
rqt_graph = subprocess.Popen(["rosrun", "rqt_graph", "rqt_graph"])

In [ ]:
os.killpg(os.getpgid(rqt_graph.pid), signal.SIGINT)

#### <span id="visual-camera">Get Live Camera Feed</span>
<span> <a href="#dataset-content"> Return to Section Contents </a> </span>

After startup in Gazebo, Baxter publishes a series of **/cameras/** topics for the 3 cameras on board Baxter:
* 'head': head_camera
* 'left': left_hand_camera
* 'right': right_hand_camera
* 'kinect': kinect raw bgr image 

These can be subscribed to by a different node to provide quasi-live feedback on Baxter's movements. In turn it is possible to superimpose visualisations, such as grasp points, to show how Baxter is operating. The feedback is not live because the asynchcronous communication between nodes does not gaurantee the receipt of data.

For more information see [Baxter Wiki](https://github.com/RethinkRobotics/sdk-docs/wiki/Using-the-Cameras)

Open a Single Camera using Identifiers above:

In [ ]:
command = "{}/scripts/open_single_camera.sh".format(os.getcwd())
visual_interpreter = subprocess.Popen(["sudo", "bash", command, "left"])

Open left, right and head camera simultaneously:

In [ ]:
command = "{}/scripts/open_all_cameras.sh".format(os.getcwd())
baxter_view = subprocess.Popen(["sudo", "bash", command])

Force the camera listener to shutdown

In [ ]:
%%bash
rosnode kill /Baxter_Cam_Listener

## <span id="process">Data Processing</span>
<a href="#content">Return to Contents</a>


### <span id="process-content">Section Contents</span>

1. <a href="#inspect">Inspect Generated Data</a>
1. <a href="#filter">Filter Generated Data</a>
    1. <a href="#filter-stage">By Stage</a>
    2. <a href="#filter-amount">By Amount</a>

### <span id="inspect">Inspect Generated Data</span>
<a href="#process-content">Return to Section Contents</a>

To help inform the filtering process, this section provides tools for inspecting the raw generated data. 

In [ ]:
data_dir = path.join(os.getcwd(),"workspace","data")
result = glob.glob(path.join(data_dir,'*.{}'.format("csv")))
csvs = [r.split(data_dir+os.path.sep)[1] for r in result]
run_ids = [csv.split("_")[0] for csv in csvs]
print("Choose a run ID: {}".format(run_ids))
run_id = input("")
while run_id not in run_ids:
    run_id = input("")
print("Inpecting contents of Run: {}".format(run_id))

#### Run Duration
Inspect how much time it took to generate the data contained in a specific run. Note that this does not work if the data has been modified or copied elsewhere since creation.

In [ ]:
def get_run_duration(run_id,data_dir):
    """Looks up iteration creation dates for a particlar run
    Returns difference between oldest and youngest iteration"""
    run_iters = [path.join(data_dir,file) for file in os.listdir(data_dir) if file.startswith("session_".format(run_id))]
    start, *middle, end = sorted(run_iters, key=path.getctime)
    time_dif = path.getctime(end)-path.getctime(start)
    return datetime.timedelta(seconds=time_dif)

print("Run Duration: {} (Hours:Minutes:Seconds)".format(get_run_duration(run_id,data_dir)))
total_time = datetime.timedelta(seconds=0)
for run in run_ids:
    total_time += get_run_duration(run,data_dir)
print("Average Duration: {} (Hours:Minutes:Seconds)".format(total_time/len(run_ids)))

#### Run Size
Inspect how much space the generated data occupies on disk. 

In [ ]:
def get_run_size(run_id,data_dir):
    """Walks iterations in a particular run and counts up their size
    Returns total size of iterations in Bytes"""
    byte_size = 0
    for root, dirs, files in os.walk(data_dir,topdown=True):
        dirs[:] = [d for d in dirs if d.startswith("session_{}".format(run_id))]
        for file in files:
            byte_size += path.getsize(path.join(root,file))
    return byte_size

def convert_size(size_bytes):
    """Converts byte to human-readable form
    Credit James Sapam 
    https://stackoverflow.com/questions/5194057/better-way-to-convert-file-sizes-in-python"""
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

print("Storage Space used: {}".format(convert_size(get_run_size(run_id,data_dir))))

total_storage = 0
for run in run_ids:
    total_storage += get_run_size(run,data_dir)
print("Average Storage Space used: {}".format(convert_size(total_storage/len(run_ids))))

#### Directory Structure
Inspect the data directory structure. For effeciency only 1 file is listed per iteration.

In [ ]:
def list_files(startpath):
    """Walk through directory and print out its structure
    Credit: Ellockie
    https://stackoverflow.com/questions/9727673/list-directory-tree-structure-in-python/49620815#49620815"""
    for root, dirs, files in os.walk(startpath,topdown=True):
        #dirs[:] = [d for d in dirs if not d.endswith("session")]
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
            break
            
list_files(path.join(os.getcwd(),"workspace","data"))

### <span id="filter">Filter Generated Data</span>
<a href="#process-content">Return to Section Contents</a>

In order to train models on the data it is necessary to perform some pre-processing. To begin, we can filter the data so that we work with subsets only.

#### <span id="filter-stage">Filter By Stage</span>
<a href="#process-content">Return to Section Contents</a>

##### Selecting Stages
Interactively choose which stage(s) to filter by. <br>
**Stages**: IDLE, SEARCH, APPROACH, GRASP, EMPTY

In [ ]:
# >>> Choose Which Stages to Include <<<
stages = ["IDLE", "SEARCH", "APPROACH", "GRASP", "EMPTY"]
print(["{}:{}".format(i,stage) for i,stage in enumerate(stages)])
resp = raw_input("Pick stage to filter by:")
chosen_stages = []
while resp:
    try:
        if 0 <= int(resp) < len(stages):
            chosen_stages.append(stages[int(resp)])
    except:
        print("Enter a number between 0 and {}")
    resp = raw_input("Chosen: {}\n ENTER to finish:".format(chosen_stages,len(stages)))
print("Final Selection: {}".format(chosen_stages))

##### Filtered Data Dictionary
Construct filtered data dictionary containing images and their MetaData

In [ ]:
# >>> Data Directory <<<
data_dir = path.join(os.getcwd(),"workspace","data")
print("Data in directory: {}".format(data_dir))

# >>> Setup <<<
data = {}
counter = 0; img_count = 0; total_count = 0;

for item in os.listdir(data_dir): 
    # >>> Images <<<
    
    if item.startswith("session"): # Folder
        # >>> Extract Identifiers <<<
        match = re.match("^session_([^_]*)_iteration_(\d+)",item)
        run,iteration = match.group(1),match.group(2)
        
        # >>> Match with MetaData <<<
        csv_file = path.join(data_dir,item,"reward.csv")
        with open(csv_file,'r') as f:
            reader = csv.DictReader(f)
            img_metadata = {}
            for row in reader:
                if row['stage'] in chosen_stages:
                    img_metadata[row['img_name']] = row
                    img_count += 1
                total_count += 1
            
        # >>> Insert into Master Dictionary <<<
        if run in data:
            if iteration in data[run]:
                data[run][iteration].update(img_metadata)
            else:
                data[run][iteration] = img_metadata          
        else:
            data[run] = {iteration:img_metadata}
        #print("Run: {}, Session: {}, Iteration: {}".format(run,item,iteration))
        
    # >>> Ground Truths <<<
    elif item.endswith(".csv"): #File
        gt_file = path.join(data_dir,item)
        with open(gt_file,"r") as f:
            reader = csv.DictReader(f)
            for row in reader:
                run,i = row["data_id"],row["iteration"]
                if run in data:
                    if i in data[run]:
                        data[run][i]["gt"] = row
                    else:
                        data[run][i] = {"gt":row}
                else: # EdgeCase
                    data[run] = {i:{"gt":row}}
print("Size of Filtered Dataset: {} out of {} total image ({:.2f}%)".format(img_count,total_count,100.0*img_count/total_count))

print(">>>Data Dictionary Structure<<<\nRunID\n\tIteration\n\t\tImage\n\t\t\tImg_MetaData\n\t\tGroundTruth\n\t\t\tGround_Truth_Data")

<b style="font-size:16pt">Data Dictionary Structure</b> <br>
Note: All keys and values are stored as Strings. 

* RunID
    * Iteration
        * Image
            * Image MetaData
        * Ground Truth
            * GroundTruth MetaData
            
<b style="font-size:14pt">Example</b>
* 'Run0'
    * '862'
        * 'img_31_t_19227.125.jpg'
            * **stage**: 'GRASP'
            * **time**: '19227.125'
            * **distance**: '0.0125309689858'
            * **prev_distance**: '0.697887540807'
            * **gripper_pos**: '[0.6335322702884931, 0.18987464581239905, 0.7978240862937698]'
            * **object_pos**: '[0.5952855203224089, 0.17787252630049952, 0.7950000546788926]'
            * **img_name**: 'img_31_t_19227.125.jpg'
            * **reward**: '54.69302275010344'
            * **visible**: 'False
        * 'gt'
            * **data_id**: 'Run0'
            * **iteration**: '862'
            * **duration**: '16.960762977600098'
            * **grasp_success**: 'False'
            * **spawned_position**: 'x: 0.572498505857\ny: 0.186026069492\nz: 0.7825'
            * **end_position**: 'x: 0.572497609673\ny: 0.186030828812\nz: 0.795007721606'
            * **spawned_orientation**: 'x: 0.0\ny: 0.0\nz: 0.051704243461168\nw: -0.9986624410721113'
            * **end_orientation**: 'x: -7.83541030138e-0\ny: -0.000116963660073\nz: -0.0517044027934\nw: 0.9986624229'
  

##### Filtered Dataset Name
Pick a unique name for the Filtered Dataset, used for creating a new folder inside the **PROJECT_DIR/workspace/data** directory. Folder will contain retino-cortical images as well as copied MetaData.

In [ ]:
dataset_name = raw_input("Choose a name for the filtered dataset, this may NOT contain the word 'session'")
filtered_path = path.join(data_dir,dataset_name)
if not path.exists(filtered_path):
    os.makedirs(filtered_path)
else:
    raise NameError("{} already exists".format(filtered_path))

##### Initialize Software Retina
May take some time. Ensure machine has GPU and is running **CUDA 10.1** with Driver **418.87.01**. 


Add Cuda path to System if not there already:

In [ ]:
output = subprocess.check_output("source scripts/switch_cuda.sh 10.1; env -0", shell=True,
                      executable="/bin/bash")
os.environ.update(line.partition('=')[::2] for line in output[46:].split('\0'))
print(output[:23])
print(dict(os.environ)["CUDA_HOME"])

Use 'nvcc --version' and 'nvidia-smi' in console to check status of GPU and CUDA.

In [ ]:
%%bash
# Specified by CUDA installer
nvcc --version
# Specified by GPU
nvidia-smi

**Start the retina**

In [ ]:
# Note: If Kernel crashes, logout of or reboot system
print("Initiliazing Software Retina...")
retina = retina.Retina(800,800)
print("Retina initialized")

##### Generate Retino-Cortical Images and MetaData
Depending on GPU, will take a long time to complete. Creates compressed versions of the original raw image files and stores them in a seperate, filtered, dataset using the name provided above.

Inside the MetaData for each image one can find the following information:

    Visibility: True (visibile) OR False (not found) // Auto-generated label is often incorrect when we get too close to the object
    Grasp: True (success) or False (failure) // Stored seperately in RUNID_ground_truths.csv
    Stage: IDLE, SEARCH, APPROACH, GRASP
    
If the dataset is filtered to only include APPROACH and GRASP stages then the problem is constrained to the pick stage of a pick-and-place pipepline. IDLE and SEARCH arguably belong under an alignment framework and hence can be trained seperately.

The "GRASP" label is published from the DataSet controller (moveBaxter) as soon as CloseGripper() is called. Hence it can accuractly correspond to the GRASPABLE state. 

However, whether the grasp succeeds or fails is determined by lifting the object back to the hover position. if the object moves with the gripper then the grasp is successful otherwise it fails. Consequently, since the GRASP state will only be active once the object is held firmly by the gripper it is difficult to assess which frames correspond to Grasp Success or Failure. One cannot easily derive this from the published GRASP stage because the amount of frames it takes to close the gripper may not be consistent. 

One rough solution to this is to conduct a manual analysis of a representative sample of iterations (see figure below). For each iteration one could record the amount of frames after the GRASP stage is published before the object is actually grasped. The maximum value of this analysis could then be used as a threshold to correctly label (most of the time) which images are in a GRASPED state. The images before this would then be in the GRASPABLE state.

<div class="row">
    <div class="col1" style="float:left">
        <img src="https://i.imgur.com/ASX0jF1.jpg" height=200px width=200px/>
        <b>GRASPABLE state</b>
    </div>
    <div class="col2" style="float:left">
        <img src="https://i.imgur.com/nJGurpe.jpg" height=200px width=200px/>
        <b>GRASPED state</b>
    </div>
</div>

Alternatively, it is possible to integrate haptic feedback into the dataset generation. That is, as soon as their is any resistance to motion in Baxter's gripper then the system is in a GRASPED state. However, this will likely lead to sparse data as it requires a 3rd subscriber. In order to match the frame, metadata and haptic feedback in an asynchronous environment it is necessary to match their timestamps. With 3 subscribers, it is likely that the timestamps will only rarely match and consequently the data may become too sparse. 

Hence, the optimal solution appears to be to merge the GRASPABLE state with the GRASPED state. This way the robot will only choose to close its grippers if the visual interpreter thinks it is likely to succeed. Otherwise the robot can either return to hover position and try again or reset the environment. 

To illustrate this problem, 4 different approaches to developing a Visual Interpreter solutuion are described below.

<p style="font-size:16pt"><b>Option A</b> (State Machine) <p>
(image only has 1 label at a time) <br>
    <b>Labels</b>: "Ready","NotReady", "Grasped", "Failed", "Empty" <br>

**Response**:
* Ready: CloseGripper
* NotReady: Approach (pick)
* Grasped: Move onto Place Task
* Empty: Search for Object
* Failed: Reset

<p style="font-size:16pt"><b>Option B</b> (Merged Graspable/Grasped) <span style="color:red">PREFERRED</span> <p>
(image only has 1 label at a time) <br>
    <b>Labels</b>: "Grasp","NotReady", "Bad", "Empty" <br>

**Response**:
* Grasp: CloseGripper if not closed already and try to place
* NotReady: Approach (pick)
* Empty: Search for Object
* Bad: Cannot grasp, reset

<p style="font-size:16pt"><b>Option C</b> (Paired Classes) <p>
(image has 3 sets of labels) <br>
    <b>Visibility</b>: Found versus Empty <br>
    <b>Grasped</b>: Grasped or NotGrasped <br> 
    <b>Graspability</b>: Ready or NotReady <br>

Main concern here is that the accuracy on Grasped versus NotGrasped will be poor because of the unreliable time taken to close the gripper.

**Response** would be based on label combination (8 possible combinations):
* Empty and NotReady and NotGrasped: Search
* Empty and Ready and NotGrasped: Search (at least 1 incorrect label)
* Empty and NotReady and Grasped: Reset (at least 1 incorrect label)
* Empty and Ready and Grasped: Try to Place (at least 1 incorrect label)
* Found and NotReady and Grasped: Try to Place (at least 1 incorrect label)
* Found and Ready and NotGrasped: CloseGripper
* Found and NotReady and NotGrasped: Approach (pick)
* Found and Ready and Grasped: Try to Place

<p style="font-size:16pt"><b>Option D</b> (Merged Graspable/Grasped & Paired Classes) <p>
(image has 2 sets of labels) <br>
    <b>Visibility</b>: Found versus Empty <br>
<b>Graspability</b>: Grasp versus NotReady versus Bad <br>

**Response** would be based on label combination (6 possible combinations)
* Found and Grasp: CloseGripper if not closed already and Try to Place
* Found and NotReady: Approach (pick)
* Found and Bad: Reset
* Empty and Bad: Reset (at least 1 incorrect label)
* Empty and NotReady: Search
* Empty and Grasp: CloseGripper if not closed already and Try to Place (at least 1 incorrect label)


In [ ]:
from enum import Enum

def get_iteration_path(run,iteration,data_dir=path.join(os.getcwd(),"workspace","data")):
    """Returns absolute path to iteration directory using its identifying attributes"""
    return path.join(data_dir,"session_{}_iteration_{}".format(run,iteration))
    
progress = IntProgress(min=0, max=img_count,description='Converting Dataset:',bar_style='success')
display(progress)

new_data = {}
for session in data.keys():
    session_data = data[session]
    for iteration in session_data.keys():
        iter_data = session_data[iteration]
        iter_path = get_iteration_path(session,iteration)
        
        for img_name in iter_data.keys():
            if img_name == 'gt': continue
            img_data = iter_data[img_name]
            if "empty" not in img_data: img_data["empty"] = "False"
            # Apply Retino-Cortical Transform
            img_path = path.join(iter_path,img_name)
            img = cv2.imread(img_path,cv2.IMREAD_COLOR)

            if img is None: 
                print("ERROR at:\n{},{},{}".format(session,iteration,img_name)); continue
                
            retina_img = retina.sample(img)
            new_img_name = "{}_{}_{}".format(session,iteration,img_name)
            new_img_path = path.join(data_dir,dataset_name,new_img_name)
            cv2.imwrite(new_img_path,retina_img)
            
            # >>> Get Label <<<
            grasp_was_successful = True if iter_data['gt']['grasp_success'] == 'True' else False 
            if img_data["empty"] in ["True","true",True]: 
                grasp_label = label.EMPTY.value
            elif img_data["stage"] == "GRASP" and grasp_was_successful:
                grasp_label = label.GRASP.value # Label: Grasp
            elif img_data["stage"] == "GRASP" and not grasp_was_successful:
                grasp_label = label.BAD.value # Label: Bad
            else:
                grasp_label = label.NOT_READY.value # Label: NotReady
            
            with open(path.join(data_dir,dataset_name,"metadata.csv"),"a") as f:
                field_names = img_data.keys() + ["label"]
                writer = csv.DictWriter(f,fieldnames=field_names)
                if f.tell() == 0:
                    writer.writeheader()
                written_data = copy.deepcopy(img_data)
                written_data["img_name"] = new_img_name
                written_data.update({"label":grasp_label})
                writer.writerow(written_data)
            progress.value += 1

#### <span id="filter-amount">Filter By Amount</span>
<a href="#process-content">Return to Section Contents</a>

Filter to build a training/testing set of a fixed size.

In [ ]:
data_dir = os.path.join(os.getcwd(),"workspace","data")
iterations = [i for i in os.listdir(data_dir) if i.startswith("session")]
random.shuffle(iterations)
resp = input("Select number of iterations out of {}".format(len(iterations)))
while True:
    if resp.isdigit():
        if 0 < int(resp) < len(iterations):
            break
    resp = input("Select number of iterations out of {}".format(len(iterations)))
    
data = np.random.choice(iterations,size=int(resp),replace=False)
print(data,len(data))

## <span id="play">Experimental Playground</span>
<a href="#content">Return to Contents</a>

This section is dedicated to experimental features. All code is tested here for robustness before it is used in the active environment. We retain examplars from successful tests here to aid understanding.

## Examplars

### Camera Recorder Factory
Used to test the parallel launching and closing of a ROS node that records images from the 'left' hand camera approximately every 0.5 seconds. 

If **poll()** returns <b style="color:green">None</b> then the node is still running. 

**wait()** ensures the next recorder is not initialized until the 1st has successfully terminated. 

**time.sleep()** simulates the execution of useful code, which may take some time to complete. In our case this would be the execution of Motion Plans for Baxter using MoveIt as we wish to record the Movement.

In [ ]:
class CameraRecorderFactory(object):
    def __init__(self,limb,start,end):
        self.limb = limb
        self.start = start
        self.end = end
        self.cameras = []
        
        
    def main_loop(self):
        for session_id in range(self.start,self.end):
            print("Starting {} recorder".format(session_id))
            recorder = subprocess.Popen(['bash', 'scripts/run_baxter_recorder.sh','left',str(session_id)])
            print("Sleeping...")
            time.sleep(2) # Wait 2 seconds (approx 4 images)
            print("Waking up!")
            print(recorder.poll())
            print("Sleeping...")
            time.sleep(2)
            print("Waking up!")
            kill = subprocess.Popen(['bash', 'scripts/kill_baxter_recorder.sh',str(session_id)])
            kill.wait(); recorder.wait() # Wait for node to shutdown cleanly
cam_factory = CameraRecorderFactory("left",2,6):
cam_factory.main_loop()

### Gripper Offset
When carrying out pick task a systematic offset in the gripper was noted. Here the appropriate corrections are found.

**Wrist Target Pose**
x=0.679, y=0.312, z=1.053, xo = -3.133, yo = 0.0 zo = -3.132

**Block Target Pose**
x=0.679, y=0.3100, z=0.820

**Block Size**: 
    0.04 x 0.04 by 0.04
    
**Correcting Offset**
x = x - 0.02
y = y + 0.02


In [ ]:
block_spawner.spawn_model(define_pose(x=0.7,y=0.4,z=0.7825))

In [ ]:
move_baxter.return_to_home_pose(open_gripper=True)

In [ ]:
move_baxter.get_diagnostics(verbose=False)

In [ ]:
move_baxter.hover_over_target(define_pose(x=0.7+offset,y=-0.1+offset,z=0.7825),pick=False)

In [ ]:
offset = 0.02
move_baxter.hover_over_target(define_pose(x=0.6799-offset,y=0.310+offset,z=0.7825),pick=False)

In [ ]:
move_baxter.move_to_target(define_pose(x=0.6799-offset,y=0.310+offset,z=0.7825))

### Frustum
The frustum defines a 3D description for the Field of View of Camera. Only objects within the frustum are visible. 
![Frustum](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/Square_frustum.png/200px-Square_frustum.png)
The Frustum is a Trapezoid which we shall approximate as a Square Pyramid. This is a reasonable assumption since the gripper ensures objects will always be at a distance from the camera.

A square pyramid can be described as follows: 

$|x| + |y| = k(h-z)$

Since the pyramid is Square $|x| == |y|$. Therefore: 

$2|x| = k(h-z)$

By placing Baxter's left arm at **x=0.0,y =1.0,z =0.64** and spawning blocks at the edges of the cameras FOV we find that: 

$|1.28|+|1.28| = k(0.64)$

Therefore for our camera:
$k = 4$

In [ ]:
pos,_ = move_baxter.get_current_pose()
bbox = move_baxter.spawner.bounding_box
resp_state = move_baxter.get_model_state("block","").pose
camera_offset = (0.03825,0.012,0.015355) # From urdf
pos[0] += camera_offset[0]
pos[1] += camera_offset[1]
pos[2] += camera_offset[2]
cam_z = pos[2]; table_z = 0.7825
x_lim = 2*(cam_z-table_z)
y_lim = 2*(cam_z-table_z)
print("Camera Pos:{}".format(pos))
print("x_lim: {}, y_lim: {}".format(x_lim,y_lim))
print(pos[0] - x_lim , abs(resp_state.position.x + bbox.x) , pos[0] + x_lim)
print(pos[0] - x_lim < abs(resp_state.position.x + bbox.x) < pos[0] + x_lim)
print(pos[0] - x_lim , abs(resp_state.position.x - bbox.x) , pos[0] + x_lim)
print(pos[0] - x_lim < abs(resp_state.position.x - bbox.x) < pos[0] + x_lim)
print(pos[1] - y_lim , abs(resp_state.position.y + bbox.y) , pos[1] + y_lim)
print(pos[1] - y_lim < abs(resp_state.position.y + bbox.y) < pos[1] + y_lim)
print(pos[1] - y_lim , abs(resp_state.position.y - bbox.y) , pos[1] + y_lim)
print(pos[1] - y_lim < abs(resp_state.position.y - bbox.y) < pos[1] + y_lim)

#### Frustum Revisited
After analysing the data, it appeared the viewing frustum was failing when objects came close to the gripper. As a result, an updated method is desired that also works well in the near-field.

First it is necessary to test the object_in_view function directly, here we look at a particular case where the viewing frustum is producing incorrect results:

In [ ]:
def object_in_view_test(grip_pos,obj_pos, table_z = 0.7825, bbox=BoundingBox(x=0.04)):

    camera_offset = (0.03825,0.012,0.015355) # From urdf
    grip_pos[0] += camera_offset[0]; grip_pos[1] += camera_offset[1]; grip_pos[2] += camera_offset[2]

    lim = grip_pos[2] - table_z
    print("Lower Bound: {:.5f}, Adjustment (+x): {:.5f}, Upper Bound: {:.5f}".format(grip_pos[0],
                      grip_pos[0]-lim,abs(obj_pos[0] + bbox.x),grip_pos[0]+lim))
    print("Lower Bound: {:.5f}, Adjustment (+y): {:.5f}, Upper Bound: {:.5f}\n".format(grip_pos[1],
                      grip_pos[1]-lim,abs(obj_pos[1] + bbox.y),grip_pos[1]+lim))
    
    print("Evaluate + Bounding Box Adjustment (x)")
    print((grip_pos[0] - lim < abs(obj_pos[0] + bbox.x) < grip_pos[0] + lim))
    print("Evaluate - Bounding Box Adjustment (x)")
    print((grip_pos[0] - lim < abs(obj_pos[0] - bbox.x) < grip_pos[0] + lim))
    print("Evaluate + Bounding Box Adjustment (y)")
    print((grip_pos[1] - lim < abs(obj_pos[1] + bbox.y) < grip_pos[1] + lim))
    print("Evaluate - Bounding Box Adjustment (y)")
    print((grip_pos[1] - lim < abs(obj_pos[1] - bbox.y) < grip_pos[1] + lim))
    
    if (((grip_pos[0] - lim < abs(obj_pos[0] + bbox.x) < grip_pos[0] + lim) or 
        (grip_pos[0] - lim < abs(obj_pos[0] - bbox.x) < grip_pos[0] + lim)) and 
        ((grip_pos[1] - lim < abs(obj_pos[1] + bbox.y) < grip_pos[1] + lim) or 
        (grip_pos[1] - lim < abs(obj_pos[1] - bbox.y) < grip_pos[1] + lim))):
        return True # In View
    else:
        return False # Not In View
    
test_data = data['Run1']['818']

for img_name in sorted(test_data):
    if img_name == "gt": continue
    
    img_data = test_data[img_name]
    grip_pos = eval(img_data["gripper_pos"])
    obj_pos = eval(img_data["object_pos"])

    default_visiblity_label = True if img_data["visible"] == 'True' else False
    
    print(img_name)
    print("\nEvaluated Positions:\nGripper {}\nObject {}\n".format(grip_pos,obj_pos))
    print("Original Verdict: {}\n".format(img_data["visible"]))
    print("\nNew Verdict: {}".format(object_in_view_test(grip_pos,obj_pos)))
    break

### Randomizing Cube Orientation
![Cube Rotation](https://upload.wikimedia.org/wikipedia/en/7/74/Cube_rotation.gif)

Our cube has 6 faces each with a different colour. If we stick to rotations of $\pi/2$ then this means there are 24 unique ways of orienting the cube.

To test this we spawn 14 cubes in a line along the surface of the table. Each one will have a random rotation about the:
* X-axis (roll)
* Y-axis (pitch)
* Z-axis (yaw)

Since a rotation of $2\pi$ is the same as a rotation of $0$ radians, it follows that there are 4 choices for each axis: <br>
\[ $0,\pi/2,\pi,3\pi/2$ \]

Spawning 14 of such random orientations at a time, we find most spawn correctly. However there are a few edge cases where the rotation is off-centre:
* $3\pi/2,\pi/2,\pi$
* $\pi,\pi/2,3\pi/2$
* $\pi/2,\pi,3\pi/2$

In [ ]:
#IEEE 754 are guaranteed to be accurate to within half a ULP
for i in range(14):
    fixed_rotations = numpy.arange(0,2*numpy.pi,numpy.pi/2)
    roll,pitch,yaw=random.choice(fixed_rotations),random.choice(fixed_rotations),random.choice(fixed_rotations)#random.uniform(0,2*numpy.pi)
    
    q =tf.transformations.quaternion_from_euler(roll,pitch,yaw)

    print(roll,pitch,yaw)
    
    bbox = move_baxter.spawner.bounding_box
    move_baxter.spawner.spawn_model(define_pose(x=1.0,y=-0.4+1.4*i*bbox.y,z=0.7825,o_x=q[0],o_y=q[1],o_z=q[2],o_w=q[3]))

From examining the document for [tf.transformations](http://docs.ros.org/jade/api/tf/html/python/transformations.html) we find that this is due to **floating point precision** errors accumulating in the conversion from euler to quaternion. 

To circumvent this, we can use quaternions directly. However, 4-dimensional quaternions are not very intuitive for humans. Hence we use the property that quaternions are non-commutative, that is we need to multiply them in a specific order to get to the desired orientation. In this case we can use this by rotating about each axis  seperately and in the correct order (zyx).

The result does not appear to generate any off-centre cubes.



In [ ]:
# Note: Floating Points (IEEE 754) are guaranteed to be accurate to within half a ULP
for i in range(14):
    fixed_rotations = numpy.arange(0,2*numpy.pi,numpy.pi/2)
    roll,pitch,yaw=random.choice(fixed_rotations),random.choice(fixed_rotations),random.choice(fixed_rotations)#random.uniform(0,2*numpy.pi)
    origin, xaxis, yaxis, zaxis = (0, 0, 0), (1, 0, 0), (0, 1, 0), (0, 0, 1)
    
    qz = quaternion_about_axis(yaw, zaxis)
    qy = quaternion_about_axis(pitch, yaxis)
    qx = quaternion_about_axis(roll, xaxis)
    
    
    q = quaternion_multiply(qz, qy)
    q = quaternion_multiply(q, qx)

    print(roll,pitch,yaw)
    
    bbox = move_baxter.spawner.bounding_box
    move_baxter.spawner.spawn_model(define_pose(x=1.0,y=-0.4+1.4*i*bbox.y,z=0.7825,o_x=q[0],o_y=q[1],o_z=q[2],o_w=q[3]))

### Run Simulationusing Subprocess
Unstable API for running Gazebo and Rviz as seperate processes (in the background). 

**Note**: All Rospy Log output is visible inside the terminal where Jupyter Notebook was launched.

In [ ]:
gazebo = Popen(['bash', 'scripts/run_gazebo.sh'])
time.sleep(2)
joint_process = Popen(['sudo','scripts/run_joint_trajectory_action_server.sh'], bufsize=1, universal_newlines=True)
time.sleep(3)
print(joint_process.poll())
# stdout_value = joint_process.communicate()[0]
# print 'Joint_Trajectory_Action_Server:\n{}'.format(stdout_value)
time.sleep(2)
rviz = Popen(['bash', 'scripts/run_rviz.sh'])

In [ ]:
gazebo2rviz = Popen(['sudo','scripts/run_model_transfer.sh'], bufsize=1, universal_newlines=True)
time.sleep(1)
print(gazebo2rviz.poll())

In [ ]:
os.killpg(os.getpgid(gazebo.pid), signal.SIGINT)
time.sleep(1)
os.killpg(os.getpgid(joint_process.pid), signal.SIGINT)
time.sleep(1)
os.killpg(os.getpgid(rviz.pid), signal.SIGINT)
time.sleep(1)
os.killpg(os.getpgid(gazebo2rviz.pid), signal.SIGINT)

### Synchronising Image Feed with Meta Data
In order to build a reliable dataset, it is necessary to associate a reward with each image from Baxter's camera feed. The challenge is that ROS's publisher-subscriber model is asynchronous, hence we cannot easily gaurantee that the image data and session metadata will be synchronised.

To get around this we can use a TimeSychnroniser, from the [Message_Filters](http://wiki.ros.org/message_filters) package. This will filter out any incoming messages that do not have matching Timestamps in their header.

However, the usual way of publishing MetaData - as a JSON parsable String - is not compatible with the message filters structure. This is because the message filter requires each of the data streams to have a header with a timestamp inside of it. To circumvent this we must define our own message type ('reward_message.msg'):

```
Header Header
string data
```
For ROS to recognise this inside of our Publisher and Subscribers, we must also modify the build script (CMAKE) to use the message_generator and message_runtime when building our custom package. The details of this can be found [here](http://wiki.ros.org/msg).


In [ ]:
from baxter_picker.msg import reward_message
from std_msgs.msg import String, Header
# JSON-parsable string
reward_str ='{{"stage":"{}","reward":{},"visible":{},"pos":{}}}'.format("IDLE",10.2,"true" if True else "false",
                                                              [3.46,2.97,6.52]) #,"visible":{},"distance":{},"prev_distance":{}}}'.format("IDLE",10.2,True,
                                                 #                           12.4,10.6)
# Define message header: ID, TimeStamp, FrameID (not needed, hence using object_name)
header = Header(24,rospy.Time.now(),"block") 
message = reward_message(header,reward_str)
print(message)

In [ ]:
import json
print(message.data)
json = json.loads(message.data)
print(json)

### Applying Retino-Cortical Transform
The Software Retina provides a biologically-inspired mechanism for compressing images. Similar to the mammalian retina, the images output of a software retina is **foveated** - that is more detail is retained at the centre of the image than its peripheries. This is implemented using a set of overlapping 2D Gaussian curves and a log-polar mapping, whose net effect is to produce a distorted image:

![Retina Image](https://i.imgur.com/Tub2cK3.jpg)

In [ ]:
img_path = "/home/xavierweiss/Automatic-Waste-Sorting/workspace/data/session_Run0_iteration_0/img_4_t_364.05.jpg"
print("Img Exists: {}".format(os.path.isfile(img_path)))
retina = retina.Retina(800,800)
print("Retina initialized")
img = cv2.imread(img_path,cv2.IMREAD_COLOR)
retina_img = retina.sample(img)
cv2.imwrite(path.join(path.expanduser("~"),"Pictures","retina_test.jpg"), retina_img);
cv2.imshow('image',retina_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Empty State
In order to train the network to respond to objects being dynamically removed, it is necessary to train an EMPTY state. This visual state exists whenever an object is not Visible. The appropriate response is thus to search for the object until it is found or a user-specified timeout passses. 

As the default training set is quite robust in that objects are successfully grasped the majority of the time, it follows that few empty states will exist. Hence, to inject more empty states into into the pipeline we spawn 'imaginary' objects and follow the normal search-approach-grasp process. However, now the gripper will close on nothing (grasp will always 'fail') and the object will never be visible.

The pick deep learning agent should not be trained with the empty state, as reward values will not be meaningful if there is no object to approach (empty state images use 0.0 as a placeholder reward).

